<a href="https://colab.research.google.com/github/jpandersen61/Machine-Learning/blob/main/ieee_xgboost_notebook_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost for IEEE-CIS Fraud Detection


In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import joblib
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve
from pandas.api.types import is_numeric_dtype

In [1]:
##Loading by using by using OS
##import os
##dataset_path = r'C:\Users\dalia\OneDrive\Desktop\PhD\Teaching (Zealand)\Tuesday\ieee-fraud-detection'

##for dirname, _, filenames in os.walk(dataset_path):
##    for filename in filenames:
##        print(os.path.join(dirname, filename))
##train_transaction = pd.read_csv(dataset_path+"/train_transaction.csv")
##train_identity = pd.read_csv(dataset_path+"/train_identity.csv")

NameError: name 'pd' is not defined

In [2]:
import pandas as pd
from pathlib import Path

from google.colab import drive

In [3]:
##Loading from Google drive
from google.colab import drive
drive.mount('/content/drive')
train_transaction = pd.read_csv("drive/My Drive/Colab Notebooks/train_transaction.csv")
train_identity = pd.read_csv("drive/My Drive/Colab Notebooks/train_identity.csv")

Mounted at /content/drive


In [4]:

df = train_transaction.merge(train_identity, on="TransactionID", how="left")
print("Train Shape:",df.shape)


Train Shape: (590540, 434)


In [5]:
#column names lower case
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace("-", "_")
df = df.sort_values(by="transactiondt")

In [6]:
target_column = 'isfraud'

features = ['transactionamt', 'productcd', 'card1', 'card2', 'card3', 'card5', 'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain',
              'c1', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14',
              'v62', 'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160', 'v165', 'v187', 'v203',
              'v207', 'v209', 'v210', 'v221', 'v234', 'v257', 'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274',
              'v277', 'v283', 'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo']

In [7]:
df.head(5)

,transactionid,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,devicetype,deviceinfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [8]:
# split time based on transactiondt. First 80% of the data is used for training and the rest for testing
train = df[df['transactiondt'] < df['transactiondt'].quantile(0.8)][features + [target_column]]
test = df[df['transactiondt'] >= df['transactiondt'].quantile(0.8)][features + [target_column]]

print("Train Shape:",train.shape)
print("Test Shape:",test.shape)

Train Shape: (472432, 68)
Test Shape: (118108, 68)
